In [ ]:
import os 
import numpy as np
import re
import matplotlib.pyplot as plt
directory_changed = False 
try:
    from simulations.inhomogeneous import simulate_inhomogeneous_record
except:
    print("Adjusting path for imports..")
    os.chdir("..")
    directory_changed = True
    from simulations.inhomogeneous import simulate_inhomogeneous_record

import results
from src.rq3 import step_inhomogeneous_spatial
from src.rq3 import _compute_cluster_sizes
import data
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing
import csv
from pathlib import Path
from datetime import datetime
print(os.getcwd())
from scriptss.parallel_sims_rq3 import worker2


# Simulating different Oak percentages on grid with a Slime distribution

In [ ]:
base_dir = Path.cwd() / "data" / "slime_experiment"
base_dir.mkdir(parents=True, exist_ok=True)

# make a new experiment directory
idx = 1
while (base_dir / f"experiment_{idx}").exists():
    idx += 1
outdir = base_dir / f"experiment_{idx}"
outdir.mkdir(parents=True, exist_ok=False)
print(f" Results are stored in: {outdir}")

In [ ]:
# --- PARAMETERS ---
L = 256             # Grid grootte (512 is mooier, 256 is sneller)
p = 0.01            # Groeisnelheid
f = 0.0001          # Bliksemfrequentie
steps = 1000        # Minimaal 5000 voor SOC evenwicht!
runs_per_param = 5  # Aantal runs per oak ratio

# We testen weer dezelfde ratios om te kunnen vergelijken met het random model
oak_ratios = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
p_burn_oak = 0.3    # Eiken branden slecht

# Lijsten bouwen
param_list = []
param_idx = 1
param_map = {} # Handig om straks resultaten terug te vinden

for ratio in oak_ratios:
    param_map[param_idx] = ratio
    
    for run_idx in range(runs_per_param):
        param_list.append({
            'L': L, 'p': p, 'f': f, 'steps': steps,
            'oak_ratio': ratio,      # Dit gaat nu naar de Slime Mold generator
            'p_burn_oak': p_burn_oak,
            'param_id': param_idx,
            'run_id': run_idx
        })
    param_idx += 1

print(f"Ready for {len(param_list)} Slime Mold simulations to run.")
print(f"Variabel Oak Ratios: {oak_ratios}")

In [ ]:
max_workers = multiprocessing.cpu_count()
print(f"🚀 Start with {max_workers} Processors...")

results = []
# Hier slaan we de data op per ratio: {0.2: [brand1, brand2...], 0.4: [...]}
fire_data_by_ratio = {ratio: [] for ratio in oak_ratios}

with ProcessPoolExecutor(max_workers=max_workers) as exe:
    # Submit alle taken aan worker2
    futures = {exe.submit(worker2, outdir, params): params for params in param_list}
    
    for fut in as_completed(futures):
        params = futures[fut]
        try:
            res = fut.result()
            
            # Welke ratio was dit?
            pid = params['param_id']
            ratio = param_map[pid]
            
            # Lees de ruwe data in vanuit de CSV die de worker heeft gemaakt
            raw_file = res.get('raw_file')
            if raw_file:
                current_fires = []
                with open(raw_file, 'r') as f:
                    reader = csv.reader(f)
                    next(reader) # Skip header
                    for row in reader:
                        if row: current_fires.append(int(row[0]))
                
                # Voeg toe aan de dataset voor deze ratio
                fire_data_by_ratio[ratio].extend(current_fires)
            
            print(f"✅ Klaar: Slime Mold Ratio {ratio:.1f}, Run {params['run_id']} -> {res['num_fires']} branden")
            results.append(res)
            
        except Exception as e:
            print(f"Error at params {params}: {e}")

print("🏁 All sims Finished")

The code below was obtained using GenAI, since I wasn't able to get the results properly from the CSV, it helped me obtain it and plot it.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

# 1. Output map
output_dir = "results/RQ3"
os.makedirs(output_dir, exist_ok=True)

plt.figure(figsize=(10, 7))
colors = plt.cm.copper(np.linspace(1, 0, len(fire_data_by_ratio)))

print("--- START DEBUG PLOT ---")

# 2. Loop
for idx, (ratio, fires) in enumerate(sorted(fire_data_by_ratio.items())):
    
    # STAP A: Converteer expliciet naar getallen (voorkomt string-fouten)
    # We gebruiken dtype=float om crashes te voorkomen, daarna int
    try:
        data = np.array(fires, dtype=float).astype(int)
    except Exception as e:
        print(f"❌ Ratio {ratio}: Kon data niet omzetten naar nummers. Fout: {e}")
        continue

    total_count = len(data)
    
    # STAP B: Filteren
    # We kijken wat er overblijft na filteren
    data_filtered = data[data > 0]
    filtered_count = len(data_filtered)
    
    print(f"Ratio {ratio:.1f}: Totaal {total_count} items. Na filter (>0): {filtered_count} items.")
    
    if filtered_count == 0:
        print(f"   ⚠️ WAARSCHUWING: Alle branden zijn grootte 0! Sla deze over.")
        # Print een voorbeeldje om te zien wat er in zit
        print(f"   Voorbeeld data (eerste 10): {fires[:10]}")
        continue
        
    # STAP C: Check Min/Max
    min_val = data_filtered.min()
    max_val = data_filtered.max()
    print(f"   Min grootte: {min_val}, Max grootte: {max_val}")

    if min_val == max_val:
        print("   ⚠️ WAARSCHUWING: Min en Max zijn gelijk (alle branden even groot). Kan geen log-lijn maken.")
        continue

    # STAP D: Bins maken
    min_pow = np.log10(min_val)
    max_pow = np.log10(max_val)
    bins = np.logspace(min_pow, max_pow, num=25)
    
    hist, edges = np.histogram(data_filtered, bins=bins, density=True)
    centers = (edges[:-1] + edges[1:]) / 2
    
    mask = hist > 0
    
    label_text = f"{int(ratio*100)}% Slime ({filtered_count} fires)"
    plt.loglog(centers[mask], hist[mask], marker='o', linestyle='-', 
               linewidth=1.5, markersize=4, color=colors[idx], 
               label=label_text, alpha=0.8)

# 3. Opmaak
plt.title(f"Fire Size Distribution: Slime Mold Model", fontsize=14)
plt.xlabel("Fire Size $S$", fontsize=12)
plt.ylabel("Frequency $P(S)$", fontsize=12)
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.4)

save_path = os.path.join(output_dir, "LogLogSlimeDistribution.png")
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"Plot opgeslagen in: {save_path}")

The code below was also obtained using GenAI, to be able to plot the heat map for the trail map (where the oaks will grow in our Slime distribution)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import os  # <--- Nodig voor mappen maken

def get_trails_for_plotting(L, ratio, steps=400):
    # 1. Instellingen
    num_agents = int(L * L * 0.08) 
    agents_x = np.random.rand(num_agents) * L
    agents_y = np.random.rand(num_agents) * L
    agents_angle = np.random.rand(num_agents) * 2 * np.pi
    trail_map = np.zeros((L, L))
    
    for _ in range(steps):
        # Move
        agents_x += np.cos(agents_angle)
        agents_y += np.sin(agents_angle)
        agents_x %= L
        agents_y %= L
        
        # Deposit
        ix = agents_x.astype(int)
        iy = agents_y.astype(int)
        np.add.at(trail_map, (ix, iy), 1.0)
        
        # Diffuse & Decay
        trail_map = gaussian_filter(trail_map, sigma=0.4) * 0.90
        
        # Sense
        agents_angle += (np.random.rand(num_agents) - 0.5) * 0.5

    return trail_map

print("Even geduld, slime mold is aan het tekenen...")

L_plot = 256
trails = get_trails_for_plotting(L=L_plot, ratio=0.4) # Ratio wordt hier niet gebruikt voor threshold, puur voor de run

# --- HET PLOTTEN EN OPSLAAN ---
# 1. Map aanmaken
output_dir = "results/RQ3"
os.makedirs(output_dir, exist_ok=True)

plt.figure(figsize=(10, 8))
plt.imshow(trails, cmap='inferno') 
plt.colorbar(label="Amount Slime (Trail Intensity)")
plt.title("The Slime Mold 'trails'\n(Light = oaks, dark = pines)")
plt.axis('off')

# 2. Opslaan (doe dit VOOR plt.show!)
save_path = os.path.join(output_dir, "slime_mold_trails_heatmap.png")
plt.savefig(save_path, dpi=300, bbox_inches='tight')

print(f"✅ Plaatje opgeslagen in: {save_path}")

# 3. Laten zien
plt.show()

The code below creates the trail maps of the random distribution alongside the Slime distribution. To show during presentation what the difference is.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import os

# 1. De Functie voor Slime Mold
def get_slime_mask(L, ratio, steps=400):
    num_agents = int(L * L * 0.08) 
    agents_x = np.random.rand(num_agents) * L
    agents_y = np.random.rand(num_agents) * L
    agents_angle = np.random.rand(num_agents) * 2 * np.pi
    trail_map = np.zeros((L, L))
    
    for _ in range(steps):
        agents_x += np.cos(agents_angle); agents_y += np.sin(agents_angle)
        agents_x %= L; agents_y %= L
        np.add.at(trail_map, (agents_x.astype(int), agents_y.astype(int)), 1.0)
        trail_map = gaussian_filter(trail_map, sigma=0.4) * 0.90 
        agents_angle += (np.random.rand(num_agents) - 0.5) * 0.5
        
    threshold = np.percentile(trail_map.flatten(), 100 - (ratio * 100))
    return trail_map > threshold

# 2. De Functie voor Random
def get_random_mask(L, ratio):
    random_noise = np.random.rand(L, L)
    return random_noise < ratio


# Map aanmaken
output_dir = "results/RQ3"
os.makedirs(output_dir, exist_ok=True)

L_plot = 256
ratio_plot = 0.4 

print(f"Genereren van kaarten met {int(ratio_plot*100)}% Eiken...")

mask_slime = get_slime_mask(L_plot, ratio_plot)
mask_random = get_random_mask(L_plot, ratio_plot)

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Random Links
axes[0].imshow(mask_random, cmap='Greens', interpolation='nearest')
axes[0].set_title(f"RANDOM Distribution ({int(ratio_plot*100)}%)\n(Noise / Scattered)", fontsize=14)
axes[0].axis('off')

# Slime Rechts
axes[1].imshow(mask_slime, cmap='Greens', interpolation='nearest')
axes[1].set_title(f"SLIME MOLD Distribution ({int(ratio_plot*100)}%)\n(Clustered / Network)", fontsize=14)
axes[1].axis('off')

plt.tight_layout()


save_path = os.path.join(output_dir, "spatial_comparison_random_vs_slime.png")
plt.savefig(save_path, dpi=300, bbox_inches='tight')

print(f"✅ Vergelijking opgeslagen in: {save_path}")

plt.show()

The code below was debugged and optimized using GenAI, so I could properly get results from two different locations and plot both the distributions (random and Slime) in one plot. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import csv
from scipy.stats import ks_2samp


folder_random = "data/inhomogeneous_experiment/experiment_7"
folder_slime  = "data/slime_experiment/experiment_15"

target_ratio = 0.4  # We willen 40% vergelijken

# We moeten even terugrekenen welke 'p' code bij 0.4 hoort voor de Slime files
# Dit is de volgorde die je in je simulatie gebruikte:
ratios_list = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

try:
    # +1 omdat param_id meestal bij 1 begint, niet 0
    slime_param_id = ratios_list.index(target_ratio) + 1 
except ValueError:
    print("⚠️ Target ratio staat niet in de standaard lijst!")
    slime_param_id = 3 # Gokje (p3 is vaak 0.4)

# De zoektermen bepalen
# Voor Random zoeken we naar "oak0.4"
search_random = f"oak{target_ratio}"
# Voor Slime zoeken we naar "_p3_" (met underscores om p13 te voorkomen)
search_slime  = f"_p{slime_param_id}_"

print(f"🔎 Strategie: Random zoeken op '{search_random}', Slime zoeken op '{search_slime}'")

# Output map
output_dir = "results/Comparison"
os.makedirs(output_dir, exist_ok=True)



def load_data_flexible(folder_path, search_string):
    fire_sizes = []
    files = glob.glob(os.path.join(folder_path, "*.csv"))
    
    found_count = 0
    for filename in files:
        if search_string in filename:
            found_count += 1
            try:
                with open(filename, 'r') as f:
                    reader = csv.reader(f)
                    header = next(reader, None) # Skip header "fire_size"
                    
                    for row in reader:
                        if row:
                            # Soms staat er rommel in, veilig omzetten
                            try:
                                val = float(row[0])
                                fire_sizes.append(int(val))
                            except:
                                continue
            except Exception as e:
                print(f"Error reading {filename}: {e}")
                
    print(f"   📂 Map: ...{folder_path[-20:]} | Term: '{search_string}' | Files: {found_count} | Data: {len(fire_sizes)}")
    return np.array(fire_sizes)


data_random = load_data_flexible(folder_random, search_random)
data_slime  = load_data_flexible(folder_slime, search_slime)

# Filter 0 en hele kleine brandjes
data_random = data_random[data_random > 0]
data_slime  = data_slime[data_slime > 0]

if len(data_random) == 0 or len(data_slime) == 0:
    print("\n❌ NOG STEEDS 0 DATA. Check de zoektermen hierboven!")
else:
    # --- 4. STATISTIEK (KS-TEST) ---
    statistic, p_value = ks_2samp(data_random, data_slime)
    
    print("\n" + "="*40)
    print(f"RESULTATEN (Ratio {target_ratio})")
    print("="*40)
    print(f"KS Statistic: {statistic:.5f}")
    print(f"P-value:      {p_value}")
    
    if p_value < 0.05:
        print("✅ SIGNIFICANT VERSCHIL! (p < 0.05)")
    else:
        print("❌ Geen significant verschil.")

    # --- 5. DE PLOT ---
    plt.figure(figsize=(10, 7))

    def get_bins(data):
        return np.logspace(np.log10(min(data)), np.log10(max(data)), 25)

    # Random (Blauw)
    bins1 = get_bins(data_random)
    hist1, edges1 = np.histogram(data_random, bins=bins1, density=True)
    centers1 = (edges1[:-1] + edges1[1:]) / 2
    plt.loglog(centers1[hist1>0], hist1[hist1>0], 'o-', label=f'Random ({target_ratio})', color='blue', alpha=0.6)

    # Slime (Rood)
    bins2 = get_bins(data_slime)
    hist2, edges2 = np.histogram(data_slime, bins=bins2, density=True)
    centers2 = (edges2[:-1] + edges2[1:]) / 2
    plt.loglog(centers2[hist2>0], hist2[hist2>0], 'o-', label=f'Slime Mold ({target_ratio})', color='red', alpha=0.6)

    plt.title(f"Random vs Slime Mold (Density {target_ratio})")
    plt.xlabel("Fire Size (S)")
    plt.ylabel("Frequency P(S)")
    plt.legend()
    plt.grid(True, which="both", ls="--", alpha=0.3)
    
    save_path = os.path.join(output_dir, f"COMPARE_final_{target_ratio}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"\n🖼️ Plot opgeslagen: {save_path}")
    plt.show()